<a href="https://colab.research.google.com/github/c444roline/dxc-ai-maintenance-assistant/blob/main/cleaning_dupontmolding_and_blowmolding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing necessary libraries
import pandas as pd
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Define the paths to the CSV files
dupont_blow_moulding_path = os.path.join('/content/drive/MyDrive/', 'Dupont_blow_moulding_manual.csv')
blow_molding_basics_path = os.path.join('/content/drive/MyDrive', 'Blow_molding_basics.csv')

# Load the CLEANED_dupont_blow_moulding.csv into a pandas DataFrame with a different encoding
if os.path.exists(dupont_blow_moulding_path):
    df_1 = pd.read_csv(dupont_blow_moulding_path, encoding='ISO-8859-1')
    print("Dupont_blow_moulding_manual.csv loaded successfully.")
else:
    print("Dupont_blow_moulding_manual.csv not found.")

# Load the Blow_molding_basics.csv into a pandas DataFrame
if os.path.exists(blow_molding_basics_path):
    df_2 = pd.read_csv(blow_molding_basics_path, encoding='ISO-8859-1')
    print("Blow_molding_basics.csv loaded successfully.")
else:
    print("Blow_molding_basics.csv not found.")


Mounted at /content/drive
Dupont_blow_moulding_manual.csv loaded successfully.
Blow_molding_basics.csv not found.


In [ ]:
#cleaning Dupont_blow_moulding_manual.csv file! (in progress)

In [ ]:
import pandas as pd
import re

print(df_1.head(10))

def extract_sections_with_indices(df):
    # Define sections and their respective subsections with starting rows
    sections_subsections = {
    "1 DuPont resins for blow moulding": [
        {"subsection": "1.1 Why blow mould engineering resins", "start_row": 4},
        {"subsection": "1.2 ZYTEL® nylon resins for blow moulding", "start_row": 5},
        {"subsection": "1.3 HYTREL® and CRASTIN® polyester resins for blow moulding", "start_row": 6}
    ],
    "2 Description of blow moulding processes": [
        {"subsection": "2.1 General", "start_row": 8},
        {"subsection": "2.2 Continuous extrusion machines", "start_row": 9},
        {"subsection": "2.3 Accumulator head machines", "start_row": 10},
        {"subsection": "2.4 Co-extrusion and sequential 3-D blow moulding", "start_row": 11},
        {"subsection": "2.5 Injection blow moulding", "start_row": 12}
    ],
    "3 The blow moulding machine – Important considerations": [
        {"subsection": "3.1 Screw and barrel design", "start_row": 14},
        {"subsection": "3.2 Manifold/adapter design", "start_row": 15},
        {"subsection": "3.3 Accumulator and continuous extrusion head design", "start_row": 16},
        {"subsection": "3.4 Die/head tooling design", "start_row": 17},
        {"subsection": "3.5 Parison cutters", "start_row": 18},
        {"subsection": "3.6 Mould clamping force", "start_row": 19},
        {"subsection": "3.7 Temperature control", "start_row": 20},
        {"subsection": "3.8 Auxiliary equipment", "start_row": 21}
    ],
    "4 Machine operating conditions": [
        {"subsection": "4.1 Quick reference – Processing conditions", "start_row": 23},
        {"subsection": "4.2 Barrel temperatures", "start_row": 24},
        {"subsection": "4.3 Adapter, head and die temperature", "start_row": 25},
        {"subsection": "4.4 Accumulator push-out pressures and speeds", "start_row": 26},
        {"subsection": "4.5 Parison programming", "start_row": 27},
        {"subsection": "4.6 Mould temperature", "start_row": 28},
        {"subsection": "4.7 Start-up procedures", "start_row": 29},
        {"subsection": "4.8 Purging and shutdown", "start_row": 30},
        {"subsection": "4.9 Secondary operations: trimming, welding", "start_row": 31},
        {"subsection": "4.10 Special conditions for injection blow moulding and 'Pressblower' operation", "start_row": 32}
    ],
    "5 Handling of blow moulding resins": [
        {"subsection": "5.1 Effects of moisture", "start_row": 34},
        {"subsection": "5.2 Drying", "start_row": 35},
        {"subsection": "5.3 Regrind", "start_row": 36},
        {"subsection": "5.4 Bulk storage", "start_row": 37}
    ],
    "6 Mould design guidance": [
        {"subsection": "6.1 General", "start_row": 39},
        {"subsection": "6.2 Materials of construction", "start_row": 40},
        {"subsection": "6.3 Blow-up (draw) ratio", "start_row": 41},
        {"subsection": "6.4 Mould shrinkage allowances and part dimensions", "start_row": 42},
        {"subsection": "6.5 Pinch-off designs", "start_row": 43},
        {"subsection": "6.6 Other mould considerations", "start_row": 44}
    ],
    "7 Troubleshooting guide": []
    }

    # Prepare to store the extracted data
    data = []

    # Iterate over each section and its subsections
    for section, subsections in sections_subsections.items():
        for i, subsection_info in enumerate(subsections):
            subsection = subsection_info["subsection"]
            start_row = subsection_info["start_row"]

            # Set end row based on the next subsection's start row or the end of the DataFrame
            if i + 1 < len(subsections):
                end_row = subsections[i + 1]["start_row"] - 1  # Use -1 to get the correct range
            else:
                end_row = len(df)  # Set to the end of the DataFrame

            # Extract text content for each subsection only from the first column
            text_block = df.iloc[start_row-1:end_row, 0].astype(str).tolist()  # Note that start_row-1 is used
            subsection_text = " ".join(text_block).strip()

            # Append the row with full section and subsection info and text
            data.append([section, subsection, subsection_text])

    # Create a structured DataFrame with section, subsection, and text columns
    organized_sections = pd.DataFrame(data, columns=['section', 'subsection', 'text'])

    return organized_sections

# Apply the function on the DataFrame
organized_sections = extract_sections_with_indices(df_1)

# Display the full organized data
organized_sections


           1.1
0          Why
1         blow
2        mould
3  engineering
4      resins?
5        There
6          are
7         many
8      reasons
9          why


,section,subsection,text
0,1 DuPont resins for blow moulding,1.1 Why blow mould engineering resins,engineering
1,1 DuPont resins for blow moulding,1.2 ZYTEL® nylon resins for blow moulding,resins?
2,1 DuPont resins for blow moulding,1.3 HYTREL® and CRASTIN® polyester resins for ...,There are many reasons why engineering resins ...
3,2 Description of blow moulding processes,2.1 General,many
4,2 Description of blow moulding processes,2.2 Continuous extrusion machines,reasons
5,2 Description of blow moulding processes,2.3 Accumulator head machines,why
6,2 Description of blow moulding processes,2.4 Co-extrusion and sequential 3-D blow moulding,engineering
7,2 Description of blow moulding processes,2.5 Injection blow moulding,resins have become established in blow mouldin...
8,3 The blow moulding machine – Important consid...,3.1 Screw and barrel design,become
9,3 The blow moulding machine – Important consid...,3.2 Manifold/adapter design,established


Not sure what this is below but i'm leaving it untouched LOL - jimena

In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=organized_sections)

https://docs.google.com/spreadsheets/d/1J-GbD33B41rdAMhbLE1_Xt4ikGUEfQ2KM6eQvwLt4D4#gid=0


/usr/local/lib/python3.10/dist-packages/google/colab/sheets.py:31: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return frame.applymap(_clean_val).replace({np.nan: None})


In [ ]:
#cleaned Blow_molding_basics.csv file! (make sure each block of information is correct)
import pandas as pd

# Function to extract sections based on predefined row indices
def extract_sections_with_indices(df):
    # Define the start rows for each section, based on your provided indices
    sections = {
        "Introduction": 3,
        "Resins": 494,
        "Processes": 712,
        "Injection Blow Moulding": 932,
        "Injection Blow-Mold Tooling": 1605,
        "Troubleshooting Injection Blow Molding": 3297,
        "Setup for Injection Blow Tooling": 3424,
        "Extrusion Blow-Molding": 3743,
        "Formulas": 5211,
        "Die/Weight Swell": 5705,
        "Heat Extraction Load": 5814,
        "Metals Used in Blow Molds": 6826,
        "Shrinkage": 6929,
        "Venting": 7130,
        "Stretch Blow Molding": 8517,
        "Other Blow-Molding Operations": 10563,
        "BIBLIOGRAPHY": 11760,
        "GENERAL REFERENCES": 12187
    }

    # Sort sections by their start rows
    sorted_sections = sorted(sections.items(), key=lambda x: x[1])

    data = []
    for i in range(len(sorted_sections)):
        section_name, start_row = sorted_sections[i]

        # Determine the end row based on the next section's start or the end of the DataFrame
        if i + 1 < len(sorted_sections):
            end_row = sorted_sections[i + 1][1]
        else:
            end_row = len(df)

        # Extract text starting from the start_row (including it)
        section_text = " ".join(df.iloc[start_row-1:end_row, 0].astype(str).tolist()).strip()

        # Append the extracted section and its content
        data.append([section_name, section_text])

    return pd.DataFrame(data, columns=['section', 'text'])

# Apply the function
organized_sections = extract_sections_with_indices(df_2)

# Display and save the organized sections DataFrame
display(organized_sections)

# Save to CSV
output_path = '/content/drive/MyDrive/organized_blow_molding_basics.csv'
organized_sections.to_csv(output_path, index=False)
print(f"File saved to {output_path}")

NameError: name 'df_2' is not defined